In [2]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_bid, close_trading
from utils import get_algod_client, wait_for_confirmation, get_balances, get_last_block_timestamp
from account import Account

In [3]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [4]:
def optin_to_asset(asset_id, sender: Account):
    txn = transaction.AssetOptInTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        index=asset_id
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    wait_for_confirmation(client, signed_txn.get_txid())

In [5]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

print(f"Creator address: {creator.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA
Seller Address: Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM


Alice is generating an example token...

In [6]:
def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
    txn = transaction.AssetConfigTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        total=total,
        decimals=decimals,
        asset_name=asset_name,
        unit_name=unit_name,
        default_frozen=False,
        strict_empty_address_check=False,
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    response = wait_for_confirmation(client, signed_txn.get_txid())
    assert response.asset_index is not None and response.asset_index > 0
    return response.asset_index

asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")
print(f"The token id is: {asset_id}")

Waiting for confirmation...
Transaction UAG2YFNRWWJW52Y5AEZI3XWQKSVAI3IVO4EOIQ2URELIU27K2DBQ confirmed in round 18481141.
The token id is: 51577182


In [7]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [8]:
appID = create_trading_app(
    client=client,
    sender=creator,
    seller=seller.get_address(),
    token_id=asset_id,
    price=price,
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction B4W57ODMXI6VYWDBZCJHWATA5PMJ3NWC4FA4D6ZJTMVUE24I6VBA confirmed in round 18481144.
App ID: 51577201
App Address: KYT2OD6W6GHLJF2TSCXJSUU62EXVWJ442M67ETVSOIPCO3PVBCW2ADBJ4U


Alice is setting up and funding token auction...

In [9]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_holder=seller,
    token_id=asset_id,
    asset_amount=1
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction CFS6E372U2EHO2B5ANHS5QR6VB7ZMC7UUYTPJ6QYPDKYEAS2YYUA confirmed in round 18481146.
Alice's algo balance:  29981000  algos


In [10]:
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))

print(f"Bidder Address: {bidder.get_address()}")

Bidder Address: GAFLYUNVHZQKQH7YLHVFNNP5WOTXDA7N3MJRVJ6SFSH7BRV4K7IJIZVAAI


In [11]:
_, last_round_time = get_last_block_timestamp(client)
if last_round_time < start_time + 5:
    sleep(start_time + 5 - last_round_time)

actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 202000, 51577182: 1}


In [12]:
bidAmount = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidAmount, " algos")

Carla wants to bid on token, her algo balance:  26991000  algos
Carla is placing bid for:  1000000  algos


In [13]:
place_bid(client=client, app_id=appID, bidder=bidder, bid_amount=bidAmount)

print("Carla is opting into token with id:", asset_id)

Waiting for confirmation...
Transaction WQ2AMBTW5XJ33LJ6MHAPPRMF6QLGKVTRDJZGEB6BNVAGMK3MRBOA confirmed in round 18481150.
Carla is opting into token with id: 51577182


In [14]:
optin_to_asset(asset_id, bidder)

Waiting for confirmation...
Transaction TKCKWH3GMTSBBXBX7XXVI2TKE4VEVTEALWLZJUHQQI2DGYJNS64A confirmed in round 18481152.


Alice is closing out the auction....

In [15]:
_, lastRoundTime = get_last_block_timestamp(client)
if lastRoundTime < end_time + 5:
    sleep(end_time + 5 - lastRoundTime)

close_trading(client, appID, seller)

51577182
b'Y4QR7LETUXJQVSPRKOJ5VFDR2GRO3MTSLJ4CLSIPJ5YN33Z3Z7HDT6JLYM'


WrongKeyBytesLengthError: key length in bytes must be 32

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 0}


In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")
actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidAmount - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 1  for token ID:  1
Alice's balances after auction:  {0: 4000024001197000, 1: 0}  Algos
Carla's balances after auction:  {0: 1000005998997000, 1: 1}  Algos
